# Experiment 008: Rotation Optimization (fix_direction)

Implement rotation optimization that finds the optimal rotation angle for each N configuration.
This can reduce the bounding box without moving trees relative to each other.

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize_scalar
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon
from tqdm import tqdm
import shutil

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    vertices = [(tx * cos_a - ty * sin_a + x, tx * sin_a + ty * cos_a + y) for tx, ty in zip(TX, TY)]
    return Polygon(vertices)

def get_all_vertices_for_n(df, n):
    """Get all vertices for configuration N"""
    prefix = f"{n:03d}_"
    trees = df[df['id'].str.startswith(prefix)]
    all_points = []
    for _, row in trees.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        poly = create_tree_polygon(x, y, deg)
        all_points.extend(list(poly.exterior.coords))
    return np.array(all_points)

def compute_bounding_side(points):
    """Compute bounding box side length"""
    if len(points) == 0:
        return 0
    return max(points.max(axis=0) - points.min(axis=0))

def optimize_rotation_for_n(df, n, angle_max=89.999):
    """Find optimal rotation angle for configuration N"""
    points = get_all_vertices_for_n(df, n)
    if len(points) < 3:
        return 0, compute_bounding_side(points)
    
    # Get convex hull for efficiency
    try:
        hull = ConvexHull(points)
        hull_points = points[hull.vertices]
    except:
        hull_points = points
    
    def bbox_side_at_angle(angle_deg):
        angle_rad = np.radians(angle_deg)
        c, s = np.cos(angle_rad), np.sin(angle_rad)
        rot_matrix = np.array([[c, s], [-s, c]])
        rotated = hull_points @ rot_matrix.T
        return max(rotated.max(axis=0) - rotated.min(axis=0))
    
    # Find optimal rotation
    result = minimize_scalar(bbox_side_at_angle, bounds=(0.001, angle_max), method='bounded')
    return result.x, result.fun

print("Functions defined")

Functions defined


In [2]:
# Load baseline
df_baseline = pd.read_csv('/home/code/external_data/saspav/santa-2025.csv')
print(f"Loaded {len(df_baseline)} rows")

# Compute baseline score
def compute_score_for_n(df, n):
    points = get_all_vertices_for_n(df, n)
    side = compute_bounding_side(points)
    return side**2 / n

baseline_score = sum(compute_score_for_n(df_baseline, n) for n in range(1, 201))
print(f"Baseline score: {baseline_score:.6f}")

Loaded 20100 rows


Baseline score: 70.659959


In [3]:
# Test rotation optimization on a few N values
print("Testing rotation optimization on sample N values...")
for n in [1, 10, 50, 100, 150, 200]:
    points = get_all_vertices_for_n(df_baseline, n)
    original_side = compute_bounding_side(points)
    opt_angle, opt_side = optimize_rotation_for_n(df_baseline, n)
    improvement = original_side - opt_side
    print(f"N={n}: original_side={original_side:.9f}, opt_angle={opt_angle:.4f}°, opt_side={opt_side:.9f}, improvement={improvement:.9f}")

Testing rotation optimization on sample N values...
N=1: original_side=0.813172798, opt_angle=89.9990°, opt_side=0.813178403, improvement=-0.000005604
N=10: original_side=1.940695856, opt_angle=0.0010°, opt_side=1.940727727, improvement=-0.000031871
N=50: original_side=4.247076254, opt_angle=89.9990°, opt_side=4.247145044, improvement=-0.000068790
N=100: original_side=5.878187599, opt_angle=0.0010°, opt_side=5.878257038, improvement=-0.000069439
N=150: original_side=7.110529230, opt_angle=0.0010°, opt_side=7.110626577, improvement=-0.000097347
N=200: original_side=8.216619493, opt_angle=0.0010°, opt_side=8.216738205, improvement=-0.000118712


In [4]:
# Apply rotation optimization to all N values
print("\nApplying rotation optimization to all N values...")
rotation_improvements = []

for n in tqdm(range(1, 201)):
    points = get_all_vertices_for_n(df_baseline, n)
    original_side = compute_bounding_side(points)
    opt_angle, opt_side = optimize_rotation_for_n(df_baseline, n)
    improvement = original_side - opt_side
    rotation_improvements.append({
        'n': n,
        'original_side': original_side,
        'opt_angle': opt_angle,
        'opt_side': opt_side,
        'improvement': improvement,
        'score_improvement': (original_side**2 - opt_side**2) / n
    })

df_improvements = pd.DataFrame(rotation_improvements)
print(f"\nTotal score improvement from rotation: {df_improvements['score_improvement'].sum():.9f}")
print(f"\nN values with positive improvement: {(df_improvements['improvement'] > 1e-9).sum()}/200")


Applying rotation optimization to all N values...


  0%|          | 0/200 [00:00<?, ?it/s]

  8%|▊         | 17/200 [00:00<00:01, 159.62it/s]

 16%|█▋        | 33/200 [00:00<00:01, 126.82it/s]

 24%|██▎       | 47/200 [00:00<00:01, 105.99it/s]

...

 96%|█████████▌| 192/200 [00:04<00:00, 28.38it/s]

 98%|█████████▊| 195/200 [00:04<00:00, 28.00it/s]

 99%|█████████▉| 198/200 [00:04<00:00, 27.64it/s]

100%|██████████| 200/200 [00:04<00:00, 46.30it/s]


Total score improvement from rotation: -0.001810491

N values with positive improvement: 0/200


In [ ]:
# Show N values with most improvement potential
print("\nTop 10 N values with most rotation improvement:")
print(df_improvements.nlargest(10, 'score_improvement')[['n', 'original_side', 'opt_angle', 'opt_side', 'score_improvement']])

In [ ]:
# Apply rotation to the configuration and save
# Note: Rotation changes the coordinates of all trees in the configuration

def apply_rotation_to_n(df, n, angle_deg):
    """Apply rotation to all trees in configuration N"""
    prefix = f"{n:03d}_"
    trees = df[df['id'].str.startswith(prefix)].copy()
    
    # Get centroid of all trees
    xs = [parse_value(row['x']) for _, row in trees.iterrows()]
    ys = [parse_value(row['y']) for _, row in trees.iterrows()]
    cx, cy = np.mean(xs), np.mean(ys)
    
    # Rotate around centroid
    angle_rad = np.radians(angle_deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    
    new_rows = []
    for _, row in trees.iterrows():
        x = parse_value(row['x']) - cx
        y = parse_value(row['y']) - cy
        deg = parse_value(row['deg'])
        
        # Rotate position
        new_x = x * cos_a - y * sin_a + cx
        new_y = x * sin_a + y * cos_a + cy
        new_deg = deg + angle_deg
        
        new_rows.append({
            'id': row['id'],
            'x': f's{new_x:.18f}',
            'y': f's{new_y:.18f}',
            'deg': f's{new_deg:.18f}'
        })
    
    return pd.DataFrame(new_rows)

# Check if rotation actually helps
if df_improvements['score_improvement'].sum() > 1e-9:
    print("Rotation provides improvement - applying to all N values...")
    
    # Build new submission with rotated configurations
    new_rows = []
    for n in tqdm(range(1, 201)):
        row_data = df_improvements[df_improvements['n'] == n].iloc[0]
        if row_data['score_improvement'] > 1e-12:
            # Apply rotation
            rotated = apply_rotation_to_n(df_baseline, n, row_data['opt_angle'])
            new_rows.append(rotated)
        else:
            # Keep original
            prefix = f"{n:03d}_"
            original = df_baseline[df_baseline['id'].str.startswith(prefix)].copy()
            new_rows.append(original)
    
    df_rotated = pd.concat(new_rows, ignore_index=True)
    print(f"Created rotated submission with {len(df_rotated)} rows")
else:
    print("No improvement from rotation - keeping baseline")
    df_rotated = df_baseline.copy()

In [ ]:
# Verify the rotated submission
def compute_total_score(df):
    total = 0
    for n in range(1, 201):
        points = get_all_vertices_for_n(df, n)
        side = compute_bounding_side(points)
        total += side**2 / n
    return total

rotated_score = compute_total_score(df_rotated)
print(f"Baseline score: {baseline_score:.6f}")
print(f"Rotated score: {rotated_score:.6f}")
print(f"Improvement: {baseline_score - rotated_score:.9f}")

In [ ]:
# Check for overlaps in rotated submission
def check_overlaps_for_n(df, n):
    prefix = f"{n:03d}_"
    trees = df[df['id'].str.startswith(prefix)]
    if len(trees) != n:
        return True, float('inf')
    
    polygons = []
    for _, row in trees.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        polygons.append(create_tree_polygon(x, y, deg))
    
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]):
                intersection = polygons[i].intersection(polygons[j])
                if intersection.area > 1e-10:
                    return True, intersection.area
    return False, 0

print("\nChecking for overlaps in rotated submission...")
overlap_count = 0
for n in range(1, 201):
    has_overlap, _ = check_overlaps_for_n(df_rotated, n)
    if has_overlap:
        overlap_count += 1
        if overlap_count <= 5:
            print(f"  N={n}: OVERLAP DETECTED")

print(f"\nTotal N values with overlaps: {overlap_count}/200")

In [ ]:
# Save submission if valid and improved
if overlap_count == 0 and rotated_score < baseline_score - 1e-9:
    df_rotated.to_csv('/home/submission/submission.csv', index=False)
    print(f"Saved rotated submission to /home/submission/submission.csv")
    print(f"Score: {rotated_score:.6f}")
elif overlap_count == 0:
    # No improvement but valid - save baseline
    shutil.copy('/home/code/external_data/saspav/santa-2025.csv', '/home/submission/submission.csv')
    print("No improvement from rotation - saved baseline")
else:
    print("Overlaps detected - not saving")

In [ ]:
# Summary
print("="*60)
print("EXPERIMENT 008 SUMMARY: Rotation Optimization")
print("="*60)
print(f"Baseline score: {baseline_score:.6f}")
print(f"Rotated score: {rotated_score:.6f}")
print(f"Improvement: {baseline_score - rotated_score:.9f}")
print(f"Overlaps: {overlap_count}/200")
print(f"\nConclusion: Rotation optimization provides {'improvement' if rotated_score < baseline_score - 1e-9 else 'NO improvement'}")
print("="*60)